### Libraries

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install eod

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import pandas as pd
import os
import re
import requests
import numpy as np
from eod import EodHistoricalData
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/onurozansunger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/onurozansunger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Script to create a DataFrame for the courses using the TSV files 

In [4]:

# Path to the folder containing TSV files
folder_path = 'HW3/courses_tsv'

dfs = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.tsv'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read TSV file into a DataFrame
        df = pd.read_csv(file_path, sep='\t', header=None, names=[
            'courseName', 'universityName', 'facultyName', 'isItFullTime',
            'description', 'startDate', 'fees', 'modality', 'duration',
            'city', 'country', 'administration', 'url'
        ])
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Write the DataFrame to a CSV file
final_df.to_csv('masters_csv', index=False)


In [5]:
df = pd.read_csv('masters_csv')
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,Future Manufacturing and Nanoscale Engineering...,University College London,Department of Mechanical Engineering,['Full Time'],Register your interest in graduate study at UC...,September,Please see the university website for further ...,MSc,1 year full time,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Marketing - MSc,University of Bristol,Faculty of Social Sciences and Law,['Full Time'],"This programme, accredited by both the Charter...",September,Please see the university website for further ...,MSc,1 year full-time,Bristol,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,Global Management of Natural Resources MSc,University College London,Department of Chemical Engineering,['Full Time'],Register your interest in graduate study at UC...,September,"Full time - £21,200",MSc,1 year full time,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
3,Business Analytics MSc,University of Stirling,"Business, Management and Law","['Full Time', 'Part Time']",Business Analytics focuses on understanding th...,"September, January",Please see the university website for further ...,"PGCert,PGDip,MSc","12 months full-time, 24 months part-time",Stirling,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
4,Public Policy and Management MSc,King’s College London,King’s Business School,"['Full Time', 'Part Time']",Whether you are a social science graduate or a...,September,Please see the university website for further ...,MSc,"1 year FT, 2 years PT",London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...


### Text Pre-processing

In [6]:
def preprocess_text(text):

    # Handling NaN/float values
    if not isinstance(text, str):
        return ''

    # Converting the text to lower case
    text = text.lower()

    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Breaking the text into words(tokens)
    words = nltk.word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Joining the list of words to form the processed text
    processed_text = ' '.join(words)

    return processed_text

### Processing the fees column

##### Getting the currencies' USD exchange rates using fastforext API


In [7]:
url = 'https://api.fastforex.io/fetch-all'
headers = {
    'accept': 'application/json',
}

# Include the API key as a query parameter
params = {
    'api_key': '56359a1bde-e4fd75ffb0-s4bxgk',
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    exchange_rates = response.json()
    

else:
    print('An error has occurred.')

exchange_rates = exchange_rates['results']

first_10_items = list(exchange_rates.items())[:10]

for key, value in first_10_items:
    print(f'{key}: {value}')


AED: 3.67261
AFN: 69.227
ALL: 95.22942
AMD: 400.88109
ANG: 1.78633
AOA: 824.89688
ARS: 353.99782
AUD: 1.53475
AWG: 1.79924
AZN: 1.69343


##### Function that retrieves the tuition fees and its currency and converts it to USD 

In [8]:
# List of currency symbols and their corresponding currency codes
currency_symbols = {
    'USD': '$',
    'EUR': '€',
    'GBP': '£',
    'JPY': '¥',
    'AUD': 'A$',
    'CAD': 'C$',
    'CHF': 'CHF',
    'CNY': '¥',
    'SEK': 'kr',
    'NZD': 'NZ$',
    'INR': '₹',
    'SGD': 'S$',
    'HKD': 'HK$',
    'NOK': 'kr',
    'KRW': '₩',
    'TRY': '₺',
    'ZAR': 'R',
    'BRL': 'R$',
    'RUB': '₽',
    'MXN': 'Mex$',
    'ARS': '$',
    'IDR': 'Rp',
    'MYR': 'RM',
    'PHP': '₱',
    'THB': '฿',
    'CZK': 'Kč',
    'HUF': 'Ft',
    'PLN': 'zł',
    'ISK': 'kr',
    'DKK': 'kr',
    'AED': 'د.إ',
    'SAR': '﷼',
    'QAR': '﷼',
    'EGP': 'EGP',
    'ILS': '₪',
    'KWD': 'د.ك',
    'BHD': 'ب.د',
    'OMR': 'ر.ع.',
    'JOD': 'د.ا',
    'LBP': 'ل.ل',
    'JMD': 'J$',
    'TTD': 'TT$',
    'COP': 'COL$',
    'CLP': 'CLP$',
    'PEN': 'S/',
    'UYU': '$U',
    'CRC': '₡',
}


def extract_and_convert_fees(row, exchange_rates):

    fees_usd = 0  
    
# Regex to find the tuition fees

    for currency, symbol in currency_symbols.items():

        pattern = rf'{symbol}(\d+(?:,\d{{3}})*(\.\d+)?)'
        matches = re.findall(pattern, row)

        for match in matches:

            # Remove commas and convert to float
            num = float(match[0].replace(',', ''))
            # Multiply by the conversion rate to get the amount in USD
            if currency in exchange_rates:
                num *= exchange_rates[currency]
                fees_usd = max(fees_usd, num)

    return fees_usd




### Applying text-preprocessing to the column 'description' and adding the column 'fees_USD'

In [9]:
df_updated = df.copy()
df_updated['description_lemmatized'] = df_updated['description'].apply(preprocess_text)
df_updated['fees_USD'] = df_updated['fees'].apply(lambda row: extract_and_convert_fees(str(row), exchange_rates) if not pd.isnull(row) else np.nan)
df_updated['fees_USD'] = df_updated['fees_USD'].round(2)

In [10]:
# Masters degrees with tuition fees mentioned in the scraped data
dummy = df_updated[df_updated['fees_USD'] != 0]
dummy.head(8)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,description_lemmatized,fees_USD
2,Global Management of Natural Resources MSc,University College London,Department of Chemical Engineering,['Full Time'],Register your interest in graduate study at UC...,September,"Full time - £21,200",MSc,1 year full time,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,regist interest graduat studi uclmor ever grow...,17011.94
6,Construction Project Management - PgDip/PgCert...,Oxford Brookes University,School of the Built Environment,"['Full Time', 'Part Time', 'Online']",Our MSc in Construction Project Management (CP...,"September, January","Home (UK) full time - £10,800 Home (UK) part t...","PGDip,MSc,PGCert","1 year full time, 2 years part time (Sept star...",Oxford,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,msc construct project manag cpm meet challeng ...,13240.42
10,Genes in Behaviour and Health (Research Master...,Vrije Universiteit Amsterdam,Faculty of Behavioural and Movement Sciences,['Full Time'],Nature and nurture: how do they interact and c...,September,"EU/EER Student: €2,209 NON-EU Student: €12,840","Research Only,MSc",2 years full time,Amsterdam,Netherlands,On Campus,https://www.findamasters.com/masters-degrees/c...,natur nurtur interact correl health wellb gene...,11764.78
12,MSc in Anthropology and Development Studies,Radboud University,Behavioural and Social Sciences,['Full Time'],How do grassroots initiatives by refugees impa...,September,The legal tuition for the 2024-2025 academic y...,MSc,1 year full time,Nijmegen,Netherlands,On Campus,https://www.findamasters.com/masters-degrees/c...,grassroot initi refuge impact upon practic pol...,16.01
13,International Real Estate and Planning MSc,University College London,The Bartlett School of Planning,"['Full Time', 'Part Time']",Register your interest in graduate study at UC...,September,"Full time - £18,000 Part time - £9,000",MSc,"1 year full time, 2 years part time, 5 years f...",London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,regist interest graduat studi uclthi programm ...,14444.10
19,MSc in Data Science & Artificial Intelligence ...,Emlyon Business School,Graduate Programmes,['Full Time'],ProgramThe aim of this program is to fill in t...,September,"€24,900 This tuition and fee charges are in ef...",MSc,18 months,Lyon,France,On Campus,https://www.findamasters.com/masters-degrees/c...,programth aim program fill gap busi engin high...,22814.87
26,Construction Management and International Deve...,University of Reading,School of Construction Management & Engineering,"['Full Time', 'Part Time']",Apply for this programmeExplore how building a...,September,"New UK/Republic of Ireland students: £12,550 ...",MSc,"1 year full time, 18-63 months flexible",Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,appli programmeexplor build infrastructur proj...,21626.03
31,Counterterrorism Risk Management and Resilienc...,Cranfield University,Cranfield Forensic Institute,"['Full Time', 'Part Time', 'Online']","OverviewThe MSc Counterterrorism, Risk Managem...",October,"• MSc Full-time £11,330 • MSc Part-time £11,33...","PGDip,PGCert,MSc",Part-time: 2 years (PgDip and PgCert) or 3 yea...,Bedford,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,overviewth msc counterterror risk manag resili...,9091.76


#### Creating the words vocabulary

In [11]:
# Create a vocabulary
vocabulary = set()

# Extract words from the 'description' column
for description in df_updated['description_lemmatized']:
     
     # Handling NaN/float values
     if isinstance(description, str):  
        words = description.split()
        vocabulary.update(words)

# Convert the vocabulary to a dictionary with words as keys and their index as values
vocabulary_dict = {idx: word for idx, word in enumerate(vocabulary)}

vocabulary_dict

{0: 'laboratorythroughout',
 1: 'essentialpleas',
 2: 'similarli',
 3: 'foundationfund',
 4: 'witha',
 5: 'imagingmed',
 6: '2017',
 7: 'thinkingw',
 8: 'boost',
 9: 'schoolthi',
 10: 'cellebritey',
 11: '2022cfa',
 12: 'proposalsth',
 13: 'comment',
 14: 'penal',
 15: '2011',
 16: 'overviewkeep',
 17: 'developmentbi',
 18: 'answersbi',
 19: 'crossdisciplinari',
 20: 'allen',
 21: 'iapt',
 22: 'examinationy',
 23: 'credibl',
 24: 'victimis',
 25: 'aacsb',
 26: 'placebas',
 27: 'controlenergi',
 28: 'forworkbas',
 29: 'mme',
 30: 'applicability·',
 31: 'macc',
 32: 'budgetsth',
 33: 'medicinethi',
 34: 'ucw',
 35: 'paiwand',
 36: 'curat',
 37: 'advisoryfinanci',
 38: 'procedurepleas',
 39: 'diabetesth',
 40: 'leagu',
 41: 'death',
 42: 'safetymanufactur',
 43: 'young',
 44: 'courseengin',
 45: 'worldwho',
 46: 'bulk',
 47: 'societyintroductionth',
 48: 'dependsin',
 49: 'workingif',
 50: 'projectoptionsustain',
 51: 'usersthi',
 52: 'mp',
 53: 'evidencedriven',
 54: 'optimisationth',
 5

#### Creating the inverted index

In [12]:
# Create a dictionary to store word occurrences with row indices
result_dict = {word: [] for word in vocabulary}

# Iterate over rows to find occurrences
i = 0
for idx, description in enumerate(df_updated['description_lemmatized']):

    # Handling NaN/float values
    if isinstance(description, str):  
        words = description.split()
        for word in words:
            if word in vocabulary:
                result_dict[word].append(idx)
    
words_frequency = {}

for index, word in vocabulary_dict.items():
    if word in result_dict:
        words_frequency[index] = result_dict[word]

words_frequency

# Create a DataFrame from the dictionary
inverted_index = pd.DataFrame(list(result_dict.items()), columns=['word', 'documents_IDs'])
inverted_index.to_csv('inverted_index.csv', index=False)
inverted_index.head()


,word,documents_IDs
0,laboratorythroughout,[5117]
1,essentialpleas,[5611]
2,similarli,"[3424, 3897]"
3,foundationfund,[1118]
4,witha,[2465]


#### Function to process the search query

In [13]:
def execute_query(query, inverted_index, df):
    
    # Apply text preprocessing to the query
    processed_query = preprocess_text(query)

    query_words = processed_query.split()

    # Find document_IDs for each word in the query using the inverted index
    result_document_ids = []
    
    for word in query_words:

        if word in inverted_index['word'].values:

            # Get the document_IDs for the current word
            word_document_ids = inverted_index.loc[inverted_index['word'] == word, 'documents_IDs'].values[0]

            # Add the document_IDs to the result list
            result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs
    result_document_ids = list(set(result_document_ids))
    result_document_ids.sort()
    
    # Return the rows corresponding to the document_IDs
    return df_updated.loc[result_document_ids, ['courseName', 'universityName', 'description', 'url']]

In [14]:
sample_query = 'advanced knowledge'
result = execute_query(sample_query, inverted_index, df_updated)
print(f'Number of results: {len(result)}')
result.head()



Number of results: 2491


,courseName,universityName,description,url
0,Future Manufacturing and Nanoscale Engineering...,University College London,Register your interest in graduate study at UC...,https://www.findamasters.com/masters-degrees/c...
1,Marketing - MSc,University of Bristol,"This programme, accredited by both the Charter...",https://www.findamasters.com/masters-degrees/c...
4,Public Policy and Management MSc,King’s College London,Whether you are a social science graduate or a...,https://www.findamasters.com/masters-degrees/c...
6,Construction Project Management - PgDip/PgCert...,Oxford Brookes University,Our MSc in Construction Project Management (CP...,https://www.findamasters.com/masters-degrees/c...
8,Construction Project Management with Industry ...,Heriot-Watt University,OverviewConstruction Project Management at Her...,https://www.findamasters.com/masters-degrees/c...
